In [ ]:
import mne  # For EEG/MEG data processing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mne.io import read_raw_edf, read_raw_fif, read_raw_bdf
from mne.preprocessing import ICA
from tqdm import tqdm
import pickle
from scipy.io import loadmat

import os

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

def ion():
    mne.viz.set_browser_backend('qt', verbose = True)
    return

def ioff():
    mne.viz.set_browser_backend('matplotlib', verbose = False)
    return

# proc_info.csv dict keys
#'SubID';'Filters';'bad_ICA';'bad_CH';'TRIG_swap';'reject';'flat';'bad_Trials';'Comment'

# Load data and basic informations

In [ ]:
# Main path that will be used for saving
main_path = '/home/lunis/Documents/nlin-EEG/data/BMD'

# Subject ID
subj = '038'

# Subject unprocessed raw header location
file_path = main_path + 'Unprocessed/subj' + subj + '.vhdr'

# Load the EEG data from the .vhdr file
raw = mne.io.read_raw_brainvision(file_path, preload=True)

# Print general information about the loaded data
print(raw.info)

# Drop Iz
raw.drop_channels(['Iz'])

In [ ]:
for ch_name, ch_type in zip(raw.info['ch_names'], raw.get_channel_types()):
    print(f"Channel: {ch_name}, Type: {ch_type}")

In [ ]:
# Sampling rate (Hz)
print(f'Sampling rate: {raw.info["sfreq"]} Hz')

# List of channels (e.g., Fp1, Fp2, Cz, etc.)
print(f'Channels: {raw.ch_names}')

# Duration of the recording (in seconds)
duration = raw.n_times / raw.info['sfreq']
print(f'Recording duration: {duration} seconds')

# Number of EEG channels
n_channels = len(raw.ch_names)
print(f'Number of channels: {n_channels}')

# Preprocessing

##  Detect long breaks in the recordings and remove them

In [ ]:
break_annots = mne.preprocessing.annotate_break(
    raw = raw,
    min_break_duration = 20,  # consider segments of at least 20 s duration
    t_start_after_previous = 2,  # start annotation 2 s after end of previous one
    t_stop_before_next = 2,  # stop annotation 2 s before beginning of next one
);

# Create complementary annotation
n_break_annots_onsets = [break_annots[i]['onset'] + break_annots[i]['duration'] for i in range(0,len(break_annots))]

n_break_annots_durations = [break_annots[i + 1]['onset'] - n_break_annots_onsets[i] for i in range(0,len(break_annots) - 1)] + [0]

# Delete segments too short for the notch filter
for i, dur in enumerate(n_break_annots_durations):

    if dur < 6.6:
        n_break_annots_onsets.pop(i)
        n_break_annots_durations.pop(i)

n_break_annots = mne.Annotations(onset = n_break_annots_onsets,
                                 duration = n_break_annots_durations,
                                 description = break_annots[0]['description'],
                                 orig_time = break_annots[0]['orig_time'],
                                 extras = None)

if len(n_break_annots) != 0:

    raw.set_annotations(raw.annotations + n_break_annots)

    raw_segments = raw.crop_by_annotations(n_break_annots)

    raw_cropped = mne.concatenate_raws(raw_segments)

    del raw_segments

else:

    raw_cropped = raw.copy()

del raw

In [ ]:
# Remove inherited annotations (There has to be a better way...)
delete = []
for i in range(0,len(raw_cropped.annotations)):

    if raw_cropped.annotations[i]['description'] == 'BAD_break':
        delete.append(i)

raw_cropped.annotations.delete(delete)
#raw_cropped.plot();

## Basic filtering and rereferencing

In [ ]:
# Filter the EEG data (remove frequencies below 1 Hz and above 40 Hz)
bandpass = [2.0,100.0]
notches = [50,100]

raw_filtered = raw_cropped.copy().filter(l_freq=bandpass[0], h_freq=bandpass[1])

# Remove AC interference
raw_filtered.notch_filter(freqs = [50,100])

In [ ]:
# Compare PRE and POST filter raw signals side by side
ioff()
pre = raw_cropped.plot(n_channels = 61, start = 250, show = False);
plt.close()
post = raw_filtered.plot(n_channels = 61, start = 250, show = False);
plt.close()

pre_canvas = FigureCanvas(pre)
pre_canvas.draw ()

pre_img = np.asarray(pre_canvas.buffer_rgba())

post_canvas = FigureCanvas(post)
post_canvas.draw ()

post_img = np.asarray(post_canvas.buffer_rgba())

fig, ax = plt.subplots(1,2, figsize = (20,20), dpi = 300)

ax[0].imshow(pre_img)
ax[1].imshow(post_img)

ax[0].set_axis_off()
ax[1].set_axis_off()

ax[0].set_title('PRE-Filtering')
ax[1].set_title('POST-Filtering')

plt.tight_layout()
plt.show()

## ICA preprocessing

In [ ]:
ica = ICA(n_components=30, random_state=42, max_iter='auto')
ica.fit(raw_filtered)

In [ ]:
### Print Power spectrum of ICA decomposition ###
sources = ica.get_sources(raw_filtered)

# Plot the power spectrum for each ICA component
n_components = ica.n_components_ 

# Loop through each ICA component
for i in range(n_components):
    # Extract the signal of component i
    component_data = sources.get_data(picks=[i])  # Get data for the i-th ICA component

    # Compute the power spectral density (PSD) of the component
    psd, freqs = mne.time_frequency.psd_array_welch(
        component_data[0],  # Extract the first row (since it's a single component)
        sfreq=raw_cropped.info['sfreq'],  # Sampling frequency from the raw data
        fmin=bandpass[0], fmax=bandpass[1],  # Focus on the filter range
        n_fft=2048,  # Length of FFT (controls frequency resolution)
        verbose = False
    )

    # Plot the power spectrum of the component
    fig, ax = plt.subplots(1,2, figsize = (7,3), dpi = 100)
    ax[1].plot(freqs, 10 * np.log10(psd), label=f'Component {i}')
    ax[1].set_xlabel('Frequency (Hz)')
    ax[1].set_ylabel('Power (dB)')
    ax[1].set_title(f'Power Spectrum of ICA Component {i}')
    ax[1].grid(True, linestyle='--', alpha=0.6)
    ax[1].set_ylim(-35,-6)

    ica.plot_components(picks = i, show_names = True, axes = ax[0]);
    plt.show()

In [ ]:
# Plot sources in interactive mode for bad sources selection
ion()
ica.plot_sources(raw_filtered, start = 250, precompute = True);

### Remove bad components

In [ ]:
# See what components where excluded in interactive mode
bad_ICA = sorted(ica.exclude)
print(bad_ICA)

In [ ]:
# Apply ICA to copy of raw data
cleaned_raw = raw_filtered.copy()
ica.apply(cleaned_raw)

In [ ]:
# Compare PRE and POST ICA raw signals side by side
ioff()
pre = raw_filtered.plot(n_channels = 61, start = 250, show = False);
plt.close()
post = cleaned_raw.plot(n_channels = 61, start = 250, show = False);
plt.close()

pre_canvas = FigureCanvas(pre)
pre_canvas.draw ()

pre_img = np.asarray(pre_canvas.buffer_rgba())

post_canvas = FigureCanvas(post)
post_canvas.draw ()

post_img = np.asarray(post_canvas.buffer_rgba())

fig, ax = plt.subplots(1,2, figsize = (20,20), dpi = 300)

ax[0].imshow(pre_img)
ax[1].imshow(post_img)

ax[0].set_axis_off()
ax[1].set_axis_off()

ax[0].set_title('PRE-ICA Removal')
ax[1].set_title('POST-ICA Removal')

plt.tight_layout()
plt.show()

In [ ]:
# Open POST ICA raw in interactive mode
ion()
cleaned_raw.plot(n_channels = 61, start = 250, precompute = True);

In [ ]:
# Save interpolated channels names and apply interpolation
bad_CH = cleaned_raw.info['bads']

for i, b in enumerate(bad_CH):
    bad_CH[i] = str(b)

if len(bad_CH) != 0:
    cleaned_raw.interpolate_bads()
    ioff()
    cleaned_raw.plot(n_channels = 61, start = 250);

    print(f'Channels {bad_CH} were interpolated')

else:
    print('No bad channels were selected')

# Extract the events

In [ ]:
# Extract event markers
events, event_dict = mne.events_from_annotations(raw_cropped) # events : n_events x ? x event code

# Print all the event names and codes
print(f'Event dictionary: {event_dict}')

# Keep only the ones we are interested in
red_event_dict = {'S__1': 1,
                  'S__2': 2,
                  'S__3': 3,
                  'S__4': 4,
                  'S_11': 11,
                  'S_12': 12,
                  'S_13': 13,
                  'S_14': 14}

# Print reduced event names and codes
print(f'Reduced event dictionary: {red_event_dict}')

In [ ]:
# Backward Maksing Triggers

# 1 Conscious Left, Self
# 2 Conscious Left, Other
# 3 Conscious Right, Self
# 4 Conscious Right, Other

#11 Unconscious Left, Self
#12 Unconscious Left, Other
#13 Unconscious Right, Self
#14 Unconscious Right, Other

#20 Correct Localization Answer (not interested)
#21 Incorrect Localization Answer (not interested)
#30 Correct Discrimination Answer (not interested)
#31 Incorrect Discrimination Answer (not interested)

#40 Invisible
#41 Almost invisible
#42 Barely Visible
#43 Visible

# Make Trial Conscious or Unconscious based on Answer

# Create copy of events array
m_events = np.copy(events)

vis_masked = 0
invis_unmasked = 0
for i, event in enumerate(events):

    # If visible
    if event[2] in [43]:
        if m_events[i - 3][2] in [11,12,13,14]:
            m_events[i - 3][2] = m_events[i - 3][2] - 10
            vis_masked += 1

    # If not visible
    elif event[2] in [40]:
        if m_events[i - 3][2] in [1,2,3,4]:
            m_events[i - 3][2] = m_events[i - 3][2] + 10
            invis_unmasked += 1

print(f'Swapped {vis_masked} masked trials which where rated conscious'
      f'\nSwapped {invis_unmasked} unmasked trials which where rated unconscious')

# Save number of swapped triggers
swap = [vis_masked,invis_unmasked]

In [ ]:
# Save Epochs file for further analysis
reject = 180e-6
flat = 100e-10

epochs = []
drops = []
for event_key in tqdm(red_event_dict.keys(), desc = 'Epoching data: ', unit = 'Condition'):

    event_id = red_event_dict[event_key]
    epoch = mne.Epochs(cleaned_raw, m_events, event_id = event_id,
                                              tmin = -0.2, tmax = 0.7,
                                              baseline = None,
                                              reject_tmin = 0.3, flat = {'eeg': flat}, reject = {'eeg': reject},
                                              preload = False, verbose = False)

    # Get number of Trials before and after dropping
    N = len(epoch.events)
    epoch.drop_bad(verbose = False)
    n = len(epoch.events)

    epochs.append(epoch)
    drops.append(N - n)

    # Save condition epoch to local
    os.makedirs(main_path + '/' + subj + '/', exist_ok = True)
    epoch.save(main_path + '/' + subj + '/' + subj + event_key + '-epo.fif', overwrite = True, verbose = False)

print('Number of dropped trials: ', drops)

In [ ]:
# Add comment regarding preprocessing results
comment = '\'Strong AC corruption\''
bandpass = [2.0,100.0]
notches = [50,100]
# Reformat SubID
fsubj = f'\'{subj}\''

filters = {'bandpass': bandpass, 'notches': notches}

# Create row to append in dataframe
sub_d = [fsubj, filters, bad_ICA, bad_CH, swap, reject, flat, drops, comment]

proc_info = pd.read_csv(main_path + '/proc_info.csv', sep = ';')
proc_info.loc[-1] = sub_d
print(proc_info)

proc_info.to_csv(main_path + '/proc_info.csv', sep = ';', index = False)

##  Check Evoked data

In [ ]:
for i, epoch in enumerate(epochs):

    mean = epoch.average()
    std = epoch.standard_error()

    fig, axs = plt.subplots(1,2, figsize = (20,7), dpi = 300)

    mean.plot(axes = axs[0], show = False)
    std.plot(axes = axs[1], show = False)

    condition = list(red_event_dict.keys())[i]

    fig.suptitle('Condition key: ' + condition)

    axs[0].set_title('Average over trials')
    axs[1].set_title('Standard Error over trials')

    plt.show()

In [ ]:
# Get exact timepoints of epochs
evoked = epochs.average()

ts = evoked.get_data()

print(f'Extracted events time series have {ts.shape[1]} time points')

#   Automated preprocessing

In [ ]:
import mne  # For EEG/MEG data processing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mne.io import read_raw_edf, read_raw_fif, read_raw_bdf
from mne.preprocessing import ICA
from tqdm import tqdm
import pickle
from scipy.io import loadmat

import os
import warnings

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

#Our Very Big Dictionary
from init import get_maind

maind = get_maind()

def ion():
    mne.viz.set_browser_backend('qt', verbose = True)
    return

def ioff():
    mne.viz.set_browser_backend('matplotlib', verbose = False)
    return

ica = ICA(n_components=30, random_state=42, max_iter='auto', verbose = False)

# Iterable processing function
def it_process(i: int):

    # Suppress warnings
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        raw = mne.io.read_raw_brainvision(instructions['file_paths'][i], preload = True, verbose = False)
        #post = mne.io.read_raw_brainvision(instructions['file_paths'][i], preload = True, verbose = False)

    # Detect breaks for removal
    break_annots = mne.preprocessing.annotate_break(
    raw = raw,
    min_break_duration = 20,  # consider segments of at least 20 s duration
    t_start_after_previous = 2,  # start annotation 2 s after end of previous one
    t_stop_before_next = 2,  # stop annotation 2 s before beginning of next one
    verbose = False)

    # Create complementary annotation
    n_break_annots_onsets = [break_annots[i]['onset'] + break_annots[i]['duration'] for i in range(0,len(break_annots))]

    n_break_annots_durations = [break_annots[i + 1]['onset'] - n_break_annots_onsets[i] for i in range(0,len(break_annots) - 1)] + [0]

    # Delete segments too short for the notch filter
    for j, dur in enumerate(n_break_annots_durations):

        if dur < 6.6:
            n_break_annots_onsets.pop(j)
            n_break_annots_durations.pop(j)

    n_break_annots = mne.Annotations(onset = n_break_annots_onsets,
                                     duration = n_break_annots_durations,
                                     description = break_annots[0]['description'],
                                     orig_time = break_annots[0]['orig_time'],
                                     extras = None)

    if len(n_break_annots) != 0:

        raw.set_annotations(raw.annotations + n_break_annots)

        raw_segments = raw.crop_by_annotations(n_break_annots)

        raw_cropped = mne.concatenate_raws(raw_segments)

        del raw_segments

    else:

        raw_cropped = raw.copy()

    del raw

    # Remove inherited annotations (There has to be a better way...)
    delete = []
    for j in range(0,len(raw_cropped.annotations)):

        if raw_cropped.annotations[j]['description'] == 'BAD_break':
            delete.append(j)

    raw_cropped.annotations.delete(delete)

    # Apply filters

    post = raw_cropped.copy()

    del raw_cropped

    # Bandpass
    post.filter(l_freq = instructions['filters']['bandpass'][0], h_freq = instructions['filters']['bandpass'][1], verbose = False);

    # Remove AC interference
    post.notch_filter(freqs = instructions['filters']['notches'], verbose = False)

    # Apply ICA if components are specified
    if instructions['ICA'][i] != None:

        ica.fit(post, verbose = False)

        ica.exclude = instructions['ICA'][i]

        ica.apply(post, verbose = False)

    # Extract event markers
    events, event_dict = mne.events_from_annotations(post, verbose = False)

    # Create copy of events array
    m_events = np.copy(events)

    swap = None

    if instructions['swap'] == True:

        vis_masked = 0
        invis_unmasked = 0
        for j, event in enumerate(events):

            # If visible
            if event[2] in [43]:
                if m_events[j - 3][2] in [11,12,13,14]:
                    m_events[j - 3][2] = m_events[j - 3][2] - 10
                    vis_masked += 1

            # If not visible
            elif event[2] in [40]:
                if m_events[j - 3][2] in [1,2,3,4]:
                    m_events[j - 3][2] = m_events[j - 3][2] + 10
                    invis_unmasked += 1

        # Save number of swapped triggers
        swap = [vis_masked,invis_unmasked]

    # Create new directory
    new_path = maind['path'] + f'data/{instructions['newcode']}/'

    # Save and drop Trials data as Epochs files
    drops = []
    for event_key in instructions['events'].keys():

        event_id = instructions['events'][event_key]
        epoch = mne.Epochs(post, m_events, event_id = event_id,
                                                tmin = -0.2, tmax = 0.7,
                                                baseline = None,
                                                reject_tmin = instructions['rejects_tmins'][i], flat = {'eeg': instructions['flats'][i]}, reject = {'eeg': instructions['rejects'][i]},
                                                preload = False, verbose = False)

        # Get number of Trials before and after dropping
        N = len(epoch.events)
        epoch.drop_bad(verbose = False)
        n = len(epoch.events)

        drops.append(N - n)

        # Save condition epoch to local
        os.makedirs(new_path + instructions['subIDs'][i] + '/', exist_ok = True)
        epoch.save(new_path + instructions['subIDs'][i] + '/' + instructions['subIDs'][i] + event_key + '-epo.fif', verbose = False, overwrite = True)

        del epoch
    del post

    # Add info to Dataframe
    comment = '\'none\''

    # Reformat SubID
    fsubj = f'\'{instructions['subIDs'][i]}\''

    # Create row to append in dataframe
    sub_d = [fsubj, instructions['filters'], instructions['ICA'][i], [], swap, instructions['rejects_tmins'][i], instructions['rejects'][i], instructions['flats'][i], drops, comment]

    #print(f'Subject {instructions['subIDs'][i]} done!')

    return sub_d

def auto_process(maind: dict, instructions: dict):

    # Unprocessed data path
    source_path = maind[instructions['exp_name']]['directories']['rw_data']

    # Subject IDs
    subIDs = maind[instructions['exp_name']]['subIDs']

    # Subject unprocessed raw header location
    file_paths = [source_path[:-1] + 'Unprocessed/subj' + subj + '.vhdr' for subj in subIDs]

    # Add lists to instructions for iteration
    instructions['subIDs'] = subIDs
    instructions['file_paths'] = file_paths

    iters = [i for i in range(0,len(subIDs))]

    from multiprocessing import Pool

    workers = instructions['MP_workers']
    chunksize = 1

    with Pool(workers, maxtasksperchild = 1) as p:
    
        results = list(tqdm(p.imap(it_process, iters, chunksize = chunksize),
                            desc = 'Processing subjects',
                            unit = 'sub',
                            total = len(iters),
                            leave = True,
                            dynamic_ncols = True)
                        )

    # Create list with processing information
    dataf = []
    for r in results:

        dataf.append(r)

    # Create new directory
    new_path = maind['path'] + f'data/{instructions['newcode']}/'

    # Save info to dataframe
    proc_info = pd.DataFrame(dataf, columns = ['SubID','Filter','bad_ICA','bad_CH','TRIG_swap', 'reject_tmin','reject', 'flat', 'bad_Trials','Comment'])

    proc_info.to_csv(new_path + 'proc_info.csv', sep = ';', index = False)

    return

##   Process without ICA

In [ ]:
# Set Instructions
instructions = {
    'MP_workers': 2,
    'exp_name': 'bmasking_dense',
    'newname': 'bmasking_dense_noICA',
    'newcode': 'BMDnoICA',
    'ch_drops': ['Iz'],
    'filters': {'bandpass': [2.0,100.0], 'notches': [50,100]},
    'ICA': [None for i in range(0,36)],
    'events': {'S__1': 1,
               'S__2': 2,
               'S__3': 3,
               'S__4': 4,
               'S_11': 11,
               'S_12': 12,
               'S_13': 13,
               'S_14': 14},
    'tlims': (-0.2,0.7),
    'show': False,
    'swap': True,
    'rejects_tmins': [0.3 for i in range(0,36)],
    'flats':[100e-10 for i in range(0,36)],
    'rejects':[300e-6 for i in range(0,36)]
}

auto_process(maind = maind, instructions = instructions)

##  Process with way too much ICA

In [ ]:
# Set Instructions
instructions = {
    'MP_workers': 2,
    'exp_name': 'bmasking_dense',
    'newname': 'bmasking_dense_highICA',
    'newcode': 'BMDhighICA',
    'ch_drops': ['Iz'],
    'filters': {'bandpass': [2.0,100.0], 'notches': [50,100]},
    'ICA': [[0,1,2,3,26,27,28,29] for i in range(0,36)],
    'events': {'S__1': 1,
               'S__2': 2,
               'S__3': 3,
               'S__4': 4,
               'S_11': 11,
               'S_12': 12,
               'S_13': 13,
               'S_14': 14},
    'tlims': (-0.2,0.7),
    'show': False,
    'swap': True,
    'rejects_tmins': [0.3 for i in range(0,36)],
    'flats':[100e-10 for i in range(0,36)],
    'rejects':[200e-6 for i in range(0,36)]
}

auto_process(maind = maind, instructions = instructions)

#   Alessio's Stuff

# Utils

In [ ]:
def match_with_tolerance(list1, list2, tol):
    """
    For each element in list1, find a unique element in list2 such that
    abs(list1[i] - list2[j]) <= tol. If multiple candidates exist,
    pick the one with minimal |difference|. Each element in list2 can
    be matched at most once.

    Returns a list `matches` of length len(list1), where matches[i] is
    the index j in list2 matched to list1[i], or None if no match.
    """
    matches = [None] * len(list1)
    used    = set()  # keep track of already-matched indices in list2

    for i, x in enumerate(list1):
        best_j    = None
        best_diff = tol + 1e-12

        # scan through list2 to find the closest unused candidate
        for j, y in enumerate(list2):
            if j in used:
                continue
            diff = abs(x - y)
            if diff <= tol and diff < best_diff:
                best_diff = diff
                best_j    = j

        if best_j is not None:
            used.add(best_j)
        matches[i] = best_j

    return matches

In [ ]:
# start_e = 1001 # code for the start of a netural trial
# start_n = 1002 # code for the start of an emotion trial
# resp_e = 1005
# resp_n = 1005
# end_e = 1007
# end_n = 1006
# confs = [41,42,43,44,51,52,53,54]
# cross_switch = 5

In [ ]:
# # neutral trial
# # The control time (the time of the switch of the cross) as to be matched with the response time of the face trials
# data = cleaned_raw
# # data = raw_filtered


# cond_codes = [(start_e, resp_e, end_e),(start_n, resp_n, end_n)]

# trials_dict = {'trial': [], 'trial_norm': [], 'baseline':[],'baseline_norm':[], 'resp_point': [], 'label': [], 'confidence':[], 'crosstime':[],
#                'matching':[]}
# for c, cond in enumerate(['real','contr']):# Loop over the two classes
#     print(cond)
#     start = cond_codes[c][0]
#     resp = cond_codes[c][1]
#     end = cond_codes[c][2]
#     where_start = np.where(events[:,2]==start)
#     for w in tqdm(where_start[0]): # Loop over the start of each trial
#         start_point = events[w][0]
#         i=1
#         itsended = False # the current trial has an end
#         control=False
#         resp_point=0
#         conf_val = None
#         control_time = None
#         while(w+i!=len(events)): # Loop over the successive sample after the start, looking for the end
#             curr_samp = events[w+i]
#             ##### DECOMMENTARE CON I TRIGGER NORMALI ####
#             # if curr_samp[2]==cross_switch and c==1:
#             #     # control = True
#             #     control_time = curr_samp[0]
#             if curr_samp[2]==start_n or curr_samp[2]==start_e: # The end of the trial is missing, discard the trial
#                 break
#             if curr_samp[2]==resp:
#                 if resp_point==0:
#                     itsended=True
#                     resp_point=curr_samp[0]
#                     end_point=curr_samp[0]
#             if curr_samp[2]==end:
#                 end_point=curr_samp[0]
#                 itsended=True
#             if curr_samp[2] in confs:
#                 conf_val = curr_samp[-1]
#                 break
#             i+=1

#         if itsended:
#             resp_point = resp_point-start_point
#             if c==0: # Face
#                 if resp_point>0:
#                     # I take the activity from -150 to -50 as a baseline (no stimulation)
#                     extracted_data = data.get_data(start=start_point-100, stop=end_point)
                    
#                     base = extracted_data[:, :100]
#                     # print(base.shape)
#                     mean_base = base.mean(axis=1)
#                     extracted_data_scale = extracted_data.T-mean_base
#                     # print(extracted_data_scale.shape)
#                     extracted_data_norm = extracted_data_scale/extracted_data_scale.std() # z-score the trial
#                     extracted_data_scale = extracted_data_scale.T
#                     extracted_data_norm = extracted_data_norm.T
                    
#                     trials_dict['trial'].append(extracted_data_scale[:, 100:])
#                     trials_dict['trial_norm'].append(extracted_data_norm[:, 100:])
#                     trials_dict['baseline'].append(extracted_data_scale[:, :100])
#                     trials_dict['baseline_norm'].append(extracted_data_norm[:, :100])
#                     trials_dict['resp_point'].append(resp_point)
#                     trials_dict['label'].append(cond)
#                     trials_dict['confidence'].append(conf_val)
#                     trials_dict['crosstime'].append(None)
#             elif c==1: # Control
#                 if resp_point>0:
#                     # I take the activity from -150 to -50 as a baseline (no stimulation)
#                     extracted_data = data.get_data(start=start_point-100, stop=end_point)
                    
#                     base = extracted_data[:, :100]
#                     # print(base.shape)
#                     mean_base = base.mean(axis=1)
#                     extracted_data_scale = extracted_data.T-mean_base
#                     # print(extracted_data_scale.shape)
#                     extracted_data_norm = extracted_data_scale/extracted_data_scale.std() # z-score the trial
#                     extracted_data_scale = extracted_data_scale.T
#                     extracted_data_norm = extracted_data_norm.T
#                     ##### DECOMMENTARE CON I TRIGGER NORMALI ####
#                     # control_time = control_time-start_point +time_adjuster
#                     # if control_time[0]>0:
#                     ##### DEINDENTARE CON I TRIGGER NORMALI #####
#                     trials_dict['trial'].append(extracted_data_scale[:, 100:])
#                     trials_dict['trial_norm'].append(extracted_data_norm[:, 100:])
#                     trials_dict['baseline'].append(extracted_data_scale[:, :100])
#                     trials_dict['baseline_norm'].append(extracted_data_norm[:, :100])
#                     trials_dict['resp_point'].append(resp_point)
#                     trials_dict['label'].append(cond)
#                     trials_dict['confidence'].append(conf_val)
#                     # trials_dict['crosstime'].append(control_time[0])
#                     trials_dict['crosstime'].append('cane')

                        
                
    
    
# print(len(trials_dict['trial']),len(trials_dict['trial_norm']),len(trials_dict['baseline']),len(trials_dict['baseline_norm']),
#       len(trials_dict['resp_point']), len(trials_dict['label']),len(trials_dict['confidence']),)
# print(trials_dict['trial'][0].shape,trials_dict['trial_norm'][0].shape,trials_dict['baseline'][0].shape,trials_dict['baseline_norm'][0].shape)
            


#------------- QUELLO VERO DA DECOMMENTARE CON I TRIGGER GIUSTI -------------------------------------#

# neutral trial
# The control time (the time of the switch of the cross) as to be matched with the response time of the face trials
data = cleaned_raw
# data = raw_filtered


cond_codes = [(start_e, resp_e, end_e),(start_n, resp_n, end_n)]

trials_dict = {'trial': [], 'trial_norm': [], 'baseline':[],'baseline_norm':[], 'resp_point': [], 'label': [], 'confidence':[], 'crosstime':[],
               'matching':[]}
for c, cond in enumerate(['real','contr']):# Loop over the two classes
    print(cond)
    start = cond_codes[c][0]
    resp = cond_codes[c][1]
    end = cond_codes[c][2]
    where_start = np.where(events[:,2]==start)
    for w in tqdm(where_start[0]): # Loop over the start of each trial
        start_point = events[w][0]
        i=1
        itsended = False # the current trial has an end
        control=False
        resp_point=0
        while(w+i!=len(events)): # Loop over the successive sample after the start, looking for the end
            curr_samp = events[w+i]
            if curr_samp[2]==cross_switch and c==1:
                # control = True
                control_time = curr_samp[0]
            if curr_samp[2]==start_n or curr_samp[2]==start_e: # The end of the trial is missing, discard the trial
                break
            if curr_samp[2]==resp:
                if resp_point==0:
                    itsended=True
                    resp_point=curr_samp[0]
                    end_point=curr_samp[0]
            if curr_samp[2]==end:
                end_point=curr_samp[0]
                itsended=True
            if curr_samp[2] in confs:
                conf_val = curr_samp[-1]
                break
            i+=1

        if itsended:
            resp_point = resp_point-start_point
            if c==0: # Face
                if resp_point>0:
                    # I take the activity from -150 to -50 as a baseline (no stimulation)
                    extracted_data = data.get_data(start=start_point-100, stop=end_point)
                    
                    base = extracted_data[:, :100]
                    # print(base.shape)
                    mean_base = base.mean(axis=1)
                    extracted_data_scale = extracted_data.T-mean_base
                    # print(extracted_data_scale.shape)
                    extracted_data_norm = extracted_data_scale/extracted_data_scale.std() # z-score the trial
                    extracted_data_scale = extracted_data_scale.T
                    extracted_data_norm = extracted_data_norm.T
                    
                    trials_dict['trial'].append(extracted_data_scale[:, 100:])
                    trials_dict['trial_norm'].append(extracted_data_norm[:, 100:])
                    trials_dict['baseline'].append(extracted_data_scale[:, :100])
                    trials_dict['baseline_norm'].append(extracted_data_norm[:, :100])
                    trials_dict['resp_point'].append(resp_point)
                    trials_dict['label'].append(cond)
                    trials_dict['confidence'].append(conf_val)
                    trials_dict['crosstime'].append(None)
            elif c==1: # Control
                if resp_point>0:
                    # I take the activity from -150 to -50 as a baseline (no stimulation)
                    extracted_data = data.get_data(start=start_point-100, stop=end_point)
                    
                    base = extracted_data[:, :100]
                    # print(base.shape)
                    mean_base = base.mean(axis=1)
                    extracted_data_scale = extracted_data.T-mean_base
                    # print(extracted_data_scale.shape)
                    extracted_data_norm = extracted_data_scale/extracted_data_scale.std() # z-score the trial
                    extracted_data_scale = extracted_data_scale.T
                    extracted_data_norm = extracted_data_norm.T
                    control_time = control_time-start_point +time_adjuster
                    # if control_time[0]>0:
                    if control_time>0:

                    
                        trials_dict['trial'].append(extracted_data_scale[:, 100:])
                        trials_dict['trial_norm'].append(extracted_data_norm[:, 100:])
                        trials_dict['baseline'].append(extracted_data_scale[:, :100])
                        trials_dict['baseline_norm'].append(extracted_data_norm[:, :100])
                        trials_dict['resp_point'].append(resp_point)
                        trials_dict['label'].append(cond)
                        trials_dict['confidence'].append(conf_val)
                        trials_dict['crosstime'].append(control_time[0])
                        # trials_dict['crosstime'].append(control_time)

                        
                
    
    
print(len(trials_dict['trial']),len(trials_dict['trial_norm']),len(trials_dict['baseline']),len(trials_dict['baseline_norm']),
      len(trials_dict['resp_point']), len(trials_dict['label']),len(trials_dict['confidence']),)
print(trials_dict['trial'][0].shape,trials_dict['trial_norm'][0].shape,trials_dict['baseline'][0].shape,trials_dict['baseline_norm'][0].shape)

In [ ]:
# # Example usage
# N_tr = np.sum(np.asarray(trials_dict['crosstime'])==None)

# list2 = np.array(np.asarray(trials_dict['resp_point'])[np.asarray(trials_dict['crosstime'])!=None]) # control
# list1 = np.asarray(trials_dict['resp_point'])[np.asarray(trials_dict['crosstime'])==None] # Face
# trials_dict['matching'] = np.full((len(list1)), -999)
# tol   = 600

# matches = match_with_tolerance(list1, list2, tol)
# for i, j in enumerate(matches):
#     if j is None:
#         print(f"list1[{i}] = {list1[i]:.2f} → no match")
#     else:
#         print(f"list1[{i}] = {list1[i]:.2f} ↔ list2[{j}] = {list2[j]:.2f}")

# for l1,l2 in enumerate(matches):
#     # print(l1,l2)
#     if l1!=None and l2!=None:
#         trials_dict['matching'][l1] = l2+N_tr
#         # trials_dict['matching'][l2+N_tr] = l1
    

# np.sum(trials_dict['matching']>=0)

#------------- QUELLO VERO DA DECOMMENTARE CON I TRIGGER GIUSTI -------------------------------------#


# Example usage
N_tr = np.sum(np.asarray(trials_dict['crosstime'])==None) # Number of face trials

list2 = np.array(np.asarray(trials_dict['resp_point'])[np.asarray(trials_dict['crosstime'])!=None]) # control
list2_rt = np.array(np.asarray(trials_dict['resp_point'])[np.asarray(trials_dict['crosstime'])!=None]) # control
list1 = np.asarray(trials_dict['resp_point'])[np.asarray(trials_dict['crosstime'])==None] # Face
trials_dict['matching'] = np.full((len(list1)), -999)
tol   = 600

matches = match_with_tolerance(list1, list2, tol)
for i, j in enumerate(matches):
    if j is None:
        print(f"list1[{i}] = {list1[i]:.2f} → no match")
    else:
        print(f"list1[{i}] = {list1[i]:.2f} ↔ list2[{j}] = {list2_rt[j]:.2f}")

for l1,l2 in enumerate(matches):
    # print(l1,l2)
    if l1!=None and l2!=None:
        trials_dict['matching'][l1] = l2+N_tr
        # trials_dict['matching'][l2+N_tr] = l1
    

np.sum(trials_dict['matching']>=0)

In [ ]:
print(trials_dict['matching'], len(trials_dict['matching']))

In [ ]:
# Print the non-matched trials and controls
print('Controls')
for i in range(N_tr):
    if i not in matches:
        print(trials_dict['crosstime'][N_tr:][i])
print('Face')
for i,m in enumerate(matches):
    if m==None:
        print(trials_dict['resp_point'][:N_tr][i])

In [ ]:
### Breaking ###
if breaking:
    with open('D:/PhD/CFS_eeg/data/new_exp/prep_task/breaking/subj_'+subj+'_trials_dict.pkl', 'wb') as handle:
        pickle.dump(trials_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Reverse ###
elif reverse:
    with open('D:/PhD/CFS_eeg/data/new_exp/prep_task/reverse/subj_'+subj+'_rev_trials_dict.pkl', 'wb') as handle:
        pickle.dump(trials_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
fig =plt.figure()
plt.hist(trials_dict['resp_point'][:142], alpha=0.4, bins=20);
plt.hist(np.array(trials_dict['crosstime'])[np.array(trials_dict['crosstime'])!=None], alpha=0.5, bins=20);
plt.show()